In [2]:
!pip install jarvis-tools

DEPRECATION: Loading egg at /home/abhijeet/miniconda3/lib/python3.12/site-packages/BoltzTraP_Tools-1.0b0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/abhijeet/miniconda3/lib/python3.12/site-packages/BoltzTraP2-25.3.1-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
#Get JARVIS structure

In [4]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms

# Get data for JARVIS ID
data = get_jid_data(jid='JVASP-125', dataset='dft_3d')

if data:
    # Create Atoms object from the data
    atoms = Atoms.from_dict(data['atoms'])
    print("Formula:", atoms.composition.reduced_formula)
    print("Structure:\n", atoms)
else:
    print("JARVIS ID not found or error accessing database")

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Formula: PtS2
Structure:
 System
1.0
1.7950132205863574 -3.109054066752567 4e-16
1.7950132205863574 3.109054066752567 -4e-16
1e-16 6e-16 5.105587861642471
Pt S 
1 2 
direct
0.0 0.0 0.0 Pt
0.6666669999999968 0.33333300000000315 0.2413708501524543 S
0.33333300000000315 0.6666669999999968 0.7586291498475456 S



In [ ]:
#Use Pymatgen to generate Niggli cell

In [24]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from pymatgen.core import Structure, Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np

def jarvis_to_pymatgen(jarvis_atoms):
    """Convert JARVIS Atoms object to pymatgen Structure"""
    lattice_mat = jarvis_atoms.lattice_mat
    lattice = Lattice(lattice_mat)
    species = jarvis_atoms.elements
    coords = jarvis_atoms.frac_coords
    struct = Structure(lattice, species, coords)
    return struct

# Get JARVIS structure
data = get_jid_data(jid='JVASP-11996', dataset='dft_3d')

if data:
    jarvis_atoms = Atoms.from_dict(data['atoms'])
    jarvis_struct = jarvis_to_pymatgen(jarvis_atoms)
    
    print("Original JARVIS structure:")
    print(f"Formula: {jarvis_struct.formula}")
    sga = SpacegroupAnalyzer(jarvis_struct)
    print(f"Space group: {sga.get_space_group_symbol()}")
    print(f"Lattice parameters: {jarvis_struct.lattice.parameters}")
    
    # CORRECT METHOD: Get primitive standardized structure (Niggli cell)
    niggli_cell = sga.get_primitive_standard_structure()
    
    print(f"\n=== Niggli Reduced Cell ===")
    print(f"Formula: {niggli_cell.formula}")
    print(f"Space group: {SpacegroupAnalyzer(niggli_cell).get_space_group_symbol()}")
    print(f"Lattice parameters: {niggli_cell.lattice.parameters}")
    print(f"Number of atoms: {len(niggli_cell)}")
    
    # Save Niggli cell as POSCAR - JUST CHANGE THIS PATH:
    niggli_cell.to(filename="/Users/abhijeet_kale/Desktop/2D_TEs/JARVIS_compare/Niggli_JVASP-11996.POSCAR")
    print(f"✓ Niggli cell saved as '/path/to/your/directory/Niggli_JVASP-11996.POSCAR'")

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Original JARVIS structure:
Formula: Hf2 Si2 Te2
Space group: P4/nmm
Lattice parameters: (3.678745268251229, 3.678745268251229, 10.018341352542956, 90.0, 90.0, 90.0)

=== Niggli Reduced Cell ===
Formula: Hf2 Si2 Te2
Space group: P4/nmm
Lattice parameters: (3.678745268251229, 3.678745268251229, 10.018341352542956, 90.0, 90.0, 90.0)
Number of atoms: 6
✓ Niggli cell saved as '/path/to/your/directory/Niggli_JVASP-11996.POSCAR'


In [ ]:
#comparison crystal structure

In [32]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from pymatgen.core import Structure, Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.structure_matcher import StructureMatcher
import os

def jarvis_to_pymatgen(jarvis_atoms):
    """Convert JARVIS Atoms object to pymatgen Structure"""
    lattice_mat = jarvis_atoms.lattice_mat
    lattice = Lattice(lattice_mat)
    species = jarvis_atoms.elements
    coords = jarvis_atoms.frac_coords
    struct = Structure(lattice, species, coords)
    return struct

def compare_structures(jarvis_struct, mc2d_struct, tolerance=0.1):
    """Compare two structures using multiple methods"""
    
    print("=" * 60)
    print("COMPARING JARVIS vs MC2D STRUCTURES")
    print("=" * 60)
    
    # Get Niggli cells for both
    sga_jarvis = SpacegroupAnalyzer(jarvis_struct)
    sga_mc2d = SpacegroupAnalyzer(mc2d_struct)
    
    niggli_jarvis = sga_jarvis.get_primitive_standard_structure()
    niggli_mc2d = sga_mc2d.get_primitive_standard_structure()
    
    print("\n=== BASIC PROPERTIES ===")
    print(f"{'Property':<20} {'JARVIS':<25} {'MC2D':<25}")
    print("-" * 70)
    print(f"{'Formula':<20} {jarvis_struct.formula:<25} {mc2d_struct.formula:<25}")
    print(f"{'Space Group':<20} {sga_jarvis.get_space_group_symbol():<25} {sga_mc2d.get_space_group_symbol():<25}")
    print(f"{'Atoms (orig)':<20} {len(jarvis_struct):<25} {len(mc2d_struct):<25}")
    print(f"{'Atoms (Niggli)':<20} {len(niggli_jarvis):<25} {len(niggli_mc2d):<25}")
    
    print("\n=== LATTICE PARAMETERS (Niggli cells) ===")
    lat_jarvis = niggli_jarvis.lattice.parameters
    lat_mc2d = niggli_mc2d.lattice.parameters
    
    print(f"{'Parameter':<10} {'JARVIS':<15} {'MC2D':<15} {'Difference':<15}")
    print("-" * 55)
    params = ['a (Å)', 'b (Å)', 'c (Å)', 'α (°)', 'β (°)', 'γ (°)']
    for i, param in enumerate(params):
        diff = abs(lat_jarvis[i] - lat_mc2d[i])
        print(f"{param:<10} {lat_jarvis[i]:<15.3f} {lat_mc2d[i]:<15.3f} {diff:<15.3f}")
    
    print("\n=== STRUCTURE MATCHING ANALYSIS ===")
    
    # Method 1: StructureMatcher (most reliable)
    matcher = StructureMatcher(ltol=tolerance, stol=tolerance, angle_tol=tolerance)
    are_identical = matcher.fit(niggli_jarvis, niggli_mc2d)
    
    print(f"StructureMatcher result: {'IDENTICAL' if are_identical else 'DIFFERENT'}")
    
    # Method 2: Compare space groups
    sg_match = sga_jarvis.get_space_group_symbol() == sga_mc2d.get_space_group_symbol()
    print(f"Space group match: {sg_match}")
    
    # Method 3: Compare formulas
    formula_match = niggli_jarvis.formula == niggli_mc2d.formula
    print(f"Chemical formula match: {formula_match}")
    
    # Method 4: Compare volumes per atom
    vol_per_atom_jarvis = niggli_jarvis.volume / len(niggli_jarvis)
    vol_per_atom_mc2d = niggli_mc2d.volume / len(niggli_mc2d)
    vol_diff = abs(vol_per_atom_jarvis - vol_per_atom_mc2d) / vol_per_atom_jarvis * 100
    
    print(f"Volume per atom: {vol_per_atom_jarvis:.3f} vs {vol_per_atom_mc2d:.3f} Å³/atom")
    print(f"Volume difference: {vol_diff:.2f}%")
    
    print("\n=== FINAL CONCLUSION ===")
    if are_identical and sg_match and formula_match and vol_diff < 5.0:
        print("✅ THE STRUCTURES ARE THE SAME!")
    else:
        print("❌ THE STRUCTURES ARE DIFFERENT")
        if not are_identical:
            print("   - StructureMatcher failed")
        if not sg_match:
            print(f"   - Different space groups: {sga_jarvis.get_space_group_symbol()} vs {sga_mc2d.get_space_group_symbol()}")
        if not formula_match:
            print(f"   - Different formulas: {niggli_jarvis.formula} vs {niggli_mc2d.formula}")
        if vol_diff >= 5.0:
            print(f"   - Significant volume difference: {vol_diff:.2f}%")
    
    return are_identical, niggli_jarvis, niggli_mc2d

# Main code
def main():
    # 1. Get JARVIS structure and convert to Niggli cell
    print("Loading JARVIS structure...")
    data = get_jid_data(jid='JVASP-4412', dataset='dft_3d')
    
    if not data:
        print("JARVIS ID not found!")
        return
    
    jarvis_atoms = Atoms.from_dict(data['atoms'])
    jarvis_struct = jarvis_to_pymatgen(jarvis_atoms)
    
    # 2. Load your MC2D structure
    print("\nLoading MC2D structure...")
    # Replace with your actual MC2D file path
    mc2d_struct = Structure.from_file("/Users/abhijeet_kale/Downloads/CuAgTe2.cif")  # Change this path!
    
    # 3. Compare the structures
    are_same, niggli_jarvis, niggli_mc2d = compare_structures(jarvis_struct, mc2d_struct)
    
    # 4. Save both Niggli cells for inspection (FIXED SYNTAX)
    niggli_jarvis.to(filename="Niggli_JARVIS.cif")
    niggli_mc2d.to(filename="Niggli_MC2D.cif")  # Fixed: removed parentheses
    print(f"\n✓ Niggli cells saved as:")
    print(f"  - Niggli_JARVIS.cif")
    print(f"  - Niggli_MC2D.cif")

if __name__ == "__main__":
    main()



Loading JARVIS structure...
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.

Loading MC2D structure...
COMPARING JARVIS vs MC2D STRUCTURES

=== BASIC PROPERTIES ===
Property             JARVIS                    MC2D                     
----------------------------------------------------------------------
Formula              Cu1 Ag1 Te2               Cu1 Ag1 Te2              
Space Group          Pmm2                      Pm                       
Atoms (orig)         4                         4                        
Atoms (Niggli)       4                         4                        

=== LATTICE PARAMETERS (Niggli cells) ===
Parameter  JARVIS          MC2D            Difference     
-------------------------------------------------------
a (Å)      3.182           4.210           1.028          
b (Å)      4.231           3.225       

/Users/abhijeet_kale/Desktop/2D_TEs/Bwann-vs-BTP2/BWann/ZrNI_Pmmn/jupyter_env/lib/python3.12/site-packages/pymatgen/io/cif.py:1029: UserWarning: No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  self.symmetry_operations = self.get_symops(data)  # type:ignore[assignment]
/Users/abhijeet_kale/Desktop/2D_TEs/Bwann-vs-BTP2/BWann/ZrNI_Pmmn/jupyter_env/lib/python3.12/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  struct = parser.parse_structures(primitive=primitive)[0]


In [ ]:
#get structures from JARVIS

In [11]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms

def print_compact_structure(jid):
    """Print crystal structure in compact format"""
    
    data = get_jid_data(jid=jid, dataset='dft_3d')
    if not data:
        print(f"JARVIS ID {jid} not found!")
        return
    
    atoms = Atoms.from_dict(data['atoms'])
    
    print("=" * 60)
    print(f"Structure: {jid} ({atoms.composition.reduced_formula})")
    print("=" * 60)
    
    # Lattice
    lengths = atoms.lattice.lat_lengths()
    angles = atoms.lattice.lat_angles()
    print(f"\nLattice Parameters:")
    print(f"  a={lengths[0]:.4f} Å, b={lengths[1]:.4f} Å, c={lengths[2]:.4f} Å")
    print(f"  α={angles[0]:.2f}°, β={angles[1]:.2f}°, γ={angles[2]:.2f}°")
    
    # Atomic positions
    print(f"\nAtomic Positions (fractional):")
    print(f"  {'Atom':<6} {'Element':<6} {'x':<8} {'y':<8} {'z':<8}")
    print(f"  {'-'*6} {'-'*6} {'-'*8} {'-'*8} {'-'*8}")
    
    for i, (element, coords) in enumerate(zip(atoms.elements, atoms.frac_coords)):
        print(f"  {i+1:<6} {element:<6} {coords[0]:<8.4f} {coords[1]:<8.4f} {coords[2]:<8.4f}")

# Quick test
print_compact_structure('JVASP-6184')

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
JARVIS ID JVASP-6184 not found!


In [ ]:
#Niggli cell comparison including C axis 

In [63]:
# ================================================================
# 0. IMPORTS
# ================================================================
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from pymatgen.core import Structure, Lattice
from pymatgen.analysis.structure_matcher import StructureMatcher
import os


# ================================================================
# 1. JARVIS → PYMATGEN CONVERTER
# ================================================================
def jarvis_to_pymatgen(jarvis_atoms):
    """
    Convert JARVIS Atoms object to pymatgen Structure.
    """
    lattice_mat = jarvis_atoms.lattice_mat
    lattice = Lattice(lattice_mat)
    species = jarvis_atoms.elements
    coords = jarvis_atoms.frac_coords
    structure = Structure(lattice, species, coords)
    return structure


# ================================================================
# 2. TRUE NIGGLI REDUCTION FUNCTION
# ================================================================
def get_true_niggli(struct):
    """
    Return a TRUE Niggli-reduced structure (mathematically unique cell).
    """
    niggli_lat = struct.lattice.get_niggli_reduced_lattice()
    niggli_struct = Structure(niggli_lat, struct.species, struct.frac_coords)
    return niggli_struct


# ================================================================
# 3. STRUCTURE COMPARISON FUNCTION
# ================================================================
def compare_structures(jarvis_struct, mc2d_struct):
    """
    Compare two crystal structures using:
      - True Niggli reduction
      - Lattice parameters
      - StructureMatcher with DEFAULT tolerances
    """

    print("=" * 70)
    print("COMPARING JARVIS vs MC2D STRUCTURES")
    print("=" * 70)

    # ------------------------------------------------------------
    # 3A. TRUE NIGGLI REDUCTION
    # ------------------------------------------------------------
    niggli_jarvis = get_true_niggli(jarvis_struct)
    niggli_mc2d = get_true_niggli(mc2d_struct)

    print("\n=== TRUE NIGGLI LATTICE PARAMETERS ===")
    params = ["a (Å)", "b (Å)", "c (Å)", "α (°)", "β (°)", "γ (°)"]

    lat_j = niggli_jarvis.lattice.parameters
    lat_m = niggli_mc2d.lattice.parameters

    print(f"{'Parameter':<10} {'JARVIS':<15} {'MC2D':<15} {'Diff':<15}")
    print("-" * 55)
    for p, lj, lm in zip(params, lat_j, lat_m):
        print(f"{p:<10} {lj:<15.4f} {lm:<15.4f} {abs(lj-lm):<15.4f}")

    # ------------------------------------------------------------
    # 3B. STRUCTURE MATCHING (DEFAULT TOLERANCES)
    # ------------------------------------------------------------
    print("\n=== STRUCTURE MATCHING (DEFAULT TOLERANCES) ===")
    print("Defaults: ltol=0.2, stol=0.3, angle_tol=5°")

    matcher = StructureMatcher()  # using DEFAULTS
    are_identical = matcher.fit(niggli_jarvis, niggli_mc2d)

    print(f"StructureMatcher result: {'IDENTICAL' if are_identical else 'DIFFERENT'}")

    # ------------------------------------------------------------
    # 3C. RETURN DATA
    # ------------------------------------------------------------
    return are_identical, niggli_jarvis, niggli_mc2d


# ================================================================
# 4. MAIN WORKFLOW
# ================================================================
def main():

    # ------------------------------------------------------------
    # 4A. LOAD JARVIS STRUCTURE
    # ------------------------------------------------------------
    print("Loading JARVIS structure...")
    data = get_jid_data(jid='JVASP-11997', dataset='dft_3d')

    if not data:
        print("ERROR: JARVIS ID not found!")
        return

    jarvis_atoms = Atoms.from_dict(data['atoms'])
    jarvis_struct = jarvis_to_pymatgen(jarvis_atoms)

    # ------------------------------------------------------------
    # 4B. LOAD MC2D STRUCTURE FROM CIF
    # ------------------------------------------------------------
    print("\nLoading MC2D structure...")
    mc2d_file = "/Users/abhijeet_kale/Downloads/As.cif"  # MODIFY PATH ACCORDINGLY
    mc2d_struct = Structure.from_file(mc2d_file)

    # ------------------------------------------------------------
    # 4C. COMPARE STRUCTURES
    # ------------------------------------------------------------
    are_same, niggli_j, niggli_m = compare_structures(jarvis_struct, mc2d_struct)

    # ------------------------------------------------------------
    # 4D. SAVE NIGGLI CELLS
    # ------------------------------------------------------------
    niggli_j.to(filename="Niggli_JARVIS.cif")
    niggli_m.to(filename="Niggli_MC2D.cif")

    print("\n✓ Saved Niggli cells:")
    print("  - Niggli_JARVIS.cif")
    print("  - Niggli_MC2D.cif")

    print("\nFinal result:", "STRUCTURES ARE SAME" if are_same else "STRUCTURES ARE DIFFERENT")


# ================================================================
# 5. EXECUTION GUARD
# ================================================================
if __name__ == "__main__":
    main()


# While parsing CIF, this code shows a trivial warning of "No _symmetry_equiv_pos_as_xyz type key" meaning no explicit symmetry operations present. 
# This commonly happens with CIFs from DFT-generated data, non-standard dataset and Wannier90 outputs.
# We can ignore this warning safely since workflow is still accurate due to folllowing three points:
# 1.Niggli reduction uses lattice, not symmetry
# 2.StructureMatcher compares atomic positions explicitly
# 3.Space groups are not needed to compare pure geometry
# ltol=0.2 (lattice length tolerance), stol=0.3 (site tolerance, 30% of avg interatomic spacing), angle_tol=5°

Loading JARVIS structure...
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.

Loading MC2D structure...
COMPARING JARVIS vs MC2D STRUCTURES

=== TRUE NIGGLI LATTICE PARAMETERS ===
Parameter  JARVIS          MC2D            Diff           
-------------------------------------------------------
a (Å)      3.7322          3.6938          0.0384         
b (Å)      4.6078          4.7194          0.1116         
c (Å)      5.9052          22.4534         16.5483        
α (°)      90.0000         90.0000         0.0000         
β (°)      108.4221        90.0000         18.4221        
γ (°)      90.0000         90.0000         0.0000         

=== STRUCTURE MATCHING (DEFAULT TOLERANCES) ===
Defaults: ltol=0.2, stol=0.3, angle_tol=5°
StructureMatcher result: DIFFERENT

✓ Saved Niggli cells:
  - Niggli_JARVIS.cif
  - Niggli_MC2D.cif

Final result: ST

In [ ]:
#exclduing c

In [49]:
# ================================================================
# FULL UPGRADED SCRIPT — NIGGLI-BASED 3D & 2D COMPARISONS (OPTION C)
# ================================================================
# Sections:
# 0. Imports
# 1. JARVIS -> pymatgen converter
# 2. Niggli reduction utilities and 2D transforms
# 3. Utilities: printing helpers and RMSD (greedy per-species)
# 4. Comparison routines (3D, c-normalized, c-fixed, 2D-invariant)
# 5. Main workflow
# ================================================================

# -------------------------
# 0. IMPORTS
# -------------------------
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from pymatgen.core import Structure, Lattice
from pymatgen.analysis.structure_matcher import StructureMatcher
import numpy as np
import math

# -------------------------
# 1. JARVIS -> PYMATGEN converter
# -------------------------
def jarvis_to_pymatgen(jarvis_atoms):
    """Convert JARVIS Atoms to pymatgen Structure."""
    lattice_mat = jarvis_atoms.lattice_mat
    lattice = Lattice(lattice_mat)
    species = jarvis_atoms.elements
    coords = jarvis_atoms.frac_coords
    return Structure(lattice, species, coords)

# -------------------------
# 2. NIGGLI REDUCTION & 2D transforms
# -------------------------
def get_true_niggli(struct):
    """Return a TRUE Niggli-reduced structure (unique canonical lattice)."""
    niggli_lat = struct.lattice.get_niggli_reduced_lattice()
    # keep fractional coordinates as-is (they map to the new lattice)
    return Structure(niggli_lat, struct.species, struct.frac_coords)

def set_c_value(struct, new_c=1.0):
    """
    Return a Structure with same a,b,angles but c set to new_c (Å).
    Fractional coordinates are kept unchanged (so absolute z scales).
    Useful for c-normalized comparisons.
    """
    a, b, c, alpha, beta, gamma = struct.lattice.parameters
    new_lat = Lattice.from_parameters(a, b, new_c, alpha, beta, gamma)
    return Structure(new_lat, struct.species, struct.frac_coords)

def drop_z_make_2d_invariant(struct):
    """
    Return a Structure that is 2D-invariant:
      - set fractional z -> 0 for all atoms
      - set c = 1.0 Å
    This compares purely in-plane geometry.
    """
    a, b, c, alpha, beta, gamma = struct.lattice.parameters
    new_lat = Lattice.from_parameters(a, b, 1.0, alpha, beta, gamma)
    new_frac_coords = [[x, y, 0.0] for (x, y, z) in struct.frac_coords]
    return Structure(new_lat, struct.species, new_frac_coords)

# -------------------------
# 3. UTILITIES
# -------------------------
def print_lattice_info(title, struct):
    """Print human-friendly lattice parameters."""
    a, b, c, alpha, beta, gamma = struct.lattice.parameters
    print(f"\n--- {title} ---")
    print(f"Lattice parameters: a={a:.6f} Å, b={b:.6f} Å, c={c:.6f} Å, "
          f"alpha={alpha:.4f}°, beta={beta:.4f}°, gamma={gamma:.4f}°")
    print(f"Num sites: {len(struct)}; Formula: {struct.formula}")

def species_counts(struct):
    """Return dictionary of species -> count."""
    counts = {}
    for s in struct.species:
        key = str(s)
        counts[key] = counts.get(key, 0) + 1
    return counts

def greedy_rmsd_by_species(struct_a, struct_b, max_distance_cutoff=5.0):
    """
    Compute a species-aware greedy nearest-neighbour RMSD between struct_a and struct_b.
    Returns (rmsd, matched_pairs_count, warning_flag)
    warning_flag=True if species sets/counts mismatch or matching suspicious.
    NOTE: This is a heuristic (fast) method; for exact assignment, use Hungarian algorithm.
    """
    # Quick checks
    counts_a = species_counts(struct_a)
    counts_b = species_counts(struct_b)
    if counts_a != counts_b:
        # species or counts differ -> can't reliably compute RMSD
        return (math.nan, 0, True)

    # Build species-wise lists of Cartesian coordinates
    cart_a = {}
    cart_b = {}
    for i, s in enumerate(struct_a.species):
        key = str(s)
        cart_a.setdefault(key, []).append(struct_a.cart_coords[i])
    for i, s in enumerate(struct_b.species):
        key = str(s)
        cart_b.setdefault(key, []).append(struct_b.cart_coords[i])

    # greedy matching per species
    d2_list = []
    total_matched = 0
    for sp in cart_a:
        pts_a = np.array(cart_a[sp])
        pts_b = np.array(cart_b[sp])
        # convert to mutable lists of points for greedy assignment
        available = pts_b.tolist()
        for pa in pts_a:
            if len(available) == 0:
                # mismatch
                return (math.nan, total_matched, True)
            # find nearest in available
            dists = np.linalg.norm(np.array(available) - pa, axis=1)
            min_idx = int(np.argmin(dists))
            min_dist = dists[min_idx]
            # optional cutoff: if too large, warn
            if min_dist > max_distance_cutoff:
                # very large distance -> suspicious
                return (math.nan, total_matched, True)
            d2_list.append(min_dist * min_dist)
            total_matched += 1
            # remove chosen
            available.pop(min_idx)

    if total_matched == 0:
        return (math.nan, 0, True)

    rmsd = math.sqrt(sum(d2_list) / total_matched)
    return (rmsd, total_matched, False)

# -------------------------
# 4. COMPARISON ROUTINES
# -------------------------
def compare_and_report(nig_j, nig_m):
    """
    All comparisons are performed on NIGGLI-REDUCED inputs (nig_j, nig_m).
    This function runs:
      - Full 3D StructureMatcher (default tolerances)
      - C-normalized match (c -> 1 Å, keep z fractional coords)
      - C-fixed 2D geometric match (c -> 1 Å, fractional z unchanged)
      - 2D-invariant match (drop z, c -> 1 Å)
    Prints explicit labeled outputs and RMSDs.
    """
    # Print Niggli cells
    print_lattice_info("NIGGLI cell (JARVIS)", nig_j)
    print_lattice_info("NIGGLI cell (MC2D) ", nig_m)

    # Prepare StructureMatcher (defaults)
    matcher = StructureMatcher()  # defaults: ltol=0.2, stol=0.3, angle_tol=5

    # -----------------------
    # 4A: Full 3D matching (NIGGLI)
    # -----------------------
    print("\n=== COMPARISON 1: NIGGLI-BASED FULL 3D MATCH ===")
    match_3d = matcher.fit(nig_j, nig_m)
    rmsd_3d, nmatches_3d, warn_3d = greedy_rmsd_by_species(nig_j, nig_m)
    print(f"StructureMatcher (3D): {'IDENTICAL' if match_3d else 'DIFFERENT'}")
    if not warn_3d:
        print(f"RMSD (3D, greedy per-species): {rmsd_3d:.4f} Å ({nmatches_3d} pairs)")
    else:
        print("RMSD (3D): cannot compute reliably (species/count mismatch or large distances)")

    # -----------------------
    # 4B: C-normalized match (keep z info but set c=1 Å)
    # -----------------------
    print("\n=== COMPARISON 2: NIGGLI-BASED C-NORMALIZED MATCH (KEEP z) ===")
    j_cnorm = set_c_value(nig_j, new_c=1.0)
    m_cnorm = set_c_value(nig_m, new_c=1.0)
    match_cnorm = matcher.fit(j_cnorm, m_cnorm)
    rmsd_cnorm, nmatches_cnorm, warn_cnorm = greedy_rmsd_by_species(j_cnorm, m_cnorm)
    print("Note: c set to 1 Å; fractional z values retained (absolute z scale changes).")
    print(f"StructureMatcher (c-normalized): {'IDENTICAL' if match_cnorm else 'DIFFERENT'}")
    if not warn_cnorm:
        print(f"RMSD (c-normalized): {rmsd_cnorm:.4f} Å ({nmatches_cnorm} pairs)")
    else:
        print("RMSD (c-normalized): cannot compute reliably")

    # -----------------------
    # 4C: C-fixed 2D geometric match (set c=1 Å, keep fractional z so z scaled)
    # (This is effectively similar to c-normalized; kept for clarity)
    # -----------------------
    print("\n=== COMPARISON 3: NIGGLI-BASED C-FIXED (2D GEOMETRIC) MATCH ===")
    # Here we re-use c-fixed structures (same as c-normalized above) but the intent is geometric 2D
    # We will also print only the in-plane lattice comparison ignoring c
    # Compare in-plane a,b and angles
    a_j, b_j, c_j, alpha_j, beta_j, gamma_j = j_cnorm.lattice.parameters
    a_m, b_m, c_m, alpha_m, beta_m, gamma_m = m_cnorm.lattice.parameters
    print(f"In-plane a,b (JARVIS vs MC2D): a={a_j:.4f} vs {a_m:.4f}, b={b_j:.4f} vs {b_m:.4f}")
    print(f"In-plane angles alpha,beta,gamma (deg): {alpha_j:.3f},{beta_j:.3f},{gamma_j:.3f}  vs  {alpha_m:.3f},{beta_m:.3f},{gamma_m:.3f}")
    match_cfixed = matcher.fit(j_cnorm, m_cnorm)
    rmsd_cfixed, nmatches_cfixed, warn_cfixed = greedy_rmsd_by_species(j_cnorm, m_cnorm)
    print(f"StructureMatcher (c-fixed 2D-geom): {'IDENTICAL' if match_cfixed else 'DIFFERENT'}")
    if not warn_cfixed:
        print(f"RMSD (c-fixed): {rmsd_cfixed:.4f} Å ({nmatches_cfixed} pairs)")
    else:
        print("RMSD (c-fixed): cannot compute reliably")

    # -----------------------
    # 4D: 2D-invariant match (drop z entirely: compare only in-plane atom ordering)
    # -----------------------
    print("\n=== COMPARISON 4: NIGGLI-BASED 2D-INVARIANT MATCH (DROP z) ===")
    j_2d = drop_z_make_2d_invariant(nig_j)
    m_2d = drop_z_make_2d_invariant(nig_m)
    # For 2D-invariant we should compare using only x,y positions: StructureMatcher still works (3rd coord is zero)
    match_2d = matcher.fit(j_2d, m_2d)
    rmsd_2d, nmatches_2d, warn_2d = greedy_rmsd_by_species(j_2d, m_2d)
    print("Note: z coordinates set to zero and c set to 1 Å; purely in-plane comparison.")
    print(f"StructureMatcher (2D-invariant): {'IDENTICAL' if match_2d else 'DIFFERENT'}")
    if not warn_2d:
        print(f"RMSD (2D-invariant): {rmsd_2d:.4f} Å ({nmatches_2d} pairs)")
    else:
        print("RMSD (2D-invariant): cannot compute reliably")

    # Summary table
    print("\n" + "="*60)
    print("SUMMARY (ALL COMPARISONS BASED ON NIGGLI-REDUCED CELLS)")
    print(f"{'Comparison':<30} {'Match':<12} {'RMSD (Å)':<12}")
    print("-"*60)
    def fmt(r, warn): return f"{r:.4f}" if (not warn and (not math.isnan(r))) else "N/A"
    print(f"{'Full 3D (Niggli)':<30} {str(match_3d):<12} {fmt(rmsd_3d, warn_3d):<12}")
    print(f"{'C-normalized (keep z)':<30} {str(match_cnorm):<12} {fmt(rmsd_cnorm, warn_cnorm):<12}")
    print(f"{'C-fixed (2D geom)':<30} {str(match_cfixed):<12} {fmt(rmsd_cfixed, warn_cfixed):<12}")
    print(f"{'2D-invariant (drop z)':<30} {str(match_2d):<12} {fmt(rmsd_2d, warn_2d):<12}")
    print("="*60)

    return {
        "match_3d": match_3d,
        "rmsd_3d": rmsd_3d,
        "match_cnorm": match_cnorm,
        "rmsd_cnorm": rmsd_cnorm,
        "match_cfixed": match_cfixed,
        "rmsd_cfixed": rmsd_cfixed,
        "match_2d": match_2d,
        "rmsd_2d": rmsd_2d,
    }

# -------------------------
# 5. MAIN WORKFLOW
# -------------------------
def main():
    # -- Inputs: change these if you like
    JARVIS_JID = "JVASP-11998"  # example; change if needed
    MC2D_CIF_PATH = "/Users/abhijeet_kale/Downloads/NbNiTe2_Pma2.cif"  # change path

    # Load JARVIS
    print("\n>> Loading JARVIS structure (jid = {})...".format(JARVIS_JID))
    data = get_jid_data(jid=JARVIS_JID, dataset="dft_3d")
    if not data:
        print("ERROR: JARVIS ID not found. Exiting.")
        return
    jarvis_atoms = Atoms.from_dict(data["atoms"])
    struct_jarvis = jarvis_to_pymatgen(jarvis_atoms)

    # Print conventional lattice info
    print_lattice_info("Conventional input (JARVIS)", struct_jarvis)

    # Load MC2D CIF
    print("\n>> Loading MC2D CIF from:", MC2D_CIF_PATH)
    struct_mc2d = Structure.from_file(MC2D_CIF_PATH)
    print_lattice_info("Conventional input (MC2D)", struct_mc2d)

    # Convert both to TRUE NIGGLI-reduced structures
    nig_j = get_true_niggli(struct_jarvis)
    nig_m = get_true_niggli(struct_mc2d)
    print("\n>> Converted both structures to TRUE NIGGLI-reduced cells (canonical).")

    # Run comparisons & report
    results = compare_and_report(nig_j, nig_m)

    # Save Niggli cells for inspection
    nig_j.to(filename="Niggli_JARVIS.cif")
    nig_m.to(filename="Niggli_MC2D.cif")
    print("\nSaved Niggli cells: Niggli_JARVIS.cif , Niggli_MC2D.cif")

    # Final verdict (conservative): require full 3D Niggli match to claim identical crystal
    if results["match_3d"]:
        print("\nFINAL VERDICT: ✅ Structures are the SAME (Niggli-based 3D match).")
    else:
        print("\nFINAL VERDICT: ❌ Structures are DIFFERENT (no Niggli-based 3D match).")
        # provide guidance from other metrics
        print("See C-normalized / c-fixed / 2D-invariant results above for 2D similarities.")

if __name__ == "__main__":
    main()


>> Loading JARVIS structure (jid = JVASP-11998)...
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.

--- Conventional input (JARVIS) ---
Lattice parameters: a=6.328766 Å, b=7.366555 Å, c=8.031459 Å, alpha=90.0000°, beta=90.0000°, gamma=90.0000°
Num sites: 16; Formula: Nb4 Ni4 Te8

>> Loading MC2D CIF from: /Users/abhijeet_kale/Downloads/NbNiTe2_Pma2.cif

--- Conventional input (MC2D) ---
Lattice parameters: a=6.293712 Å, b=8.001129 Å, c=24.716521 Å, alpha=90.0000°, beta=90.0000°, gamma=90.0000°
Num sites: 16; Formula: Nb4 Ni4 Te8

>> Converted both structures to TRUE NIGGLI-reduced cells (canonical).

--- NIGGLI cell (JARVIS) ---
Lattice parameters: a=6.328766 Å, b=7.366555 Å, c=8.031459 Å, alpha=90.0000°, beta=90.0000°, gamma=90.0000°
Num sites: 16; Formula: Nb4 Ni4 Te8

--- NIGGLI cell (MC2D)  ---
Lattice parameters: a=6.293712 Å, b=8.001129 Å